# RAG multi-modal avec LangChain

## Installation 

In [ ]:
# %sudo apt-get install poppler-utils tesseract-ocr libmagic-dev

In [ ]:
import os


os.environ["OPENAI_API_KEY"] = "sk-..."
os.environ["GROQ_API_KEY"] = "sk-..."
os.environ["LANGCHAIN_API_KEY"] = "sk-..."
os.environ["LANGCHAIN_TRACING_V2"] = "true"


## Extraction des données 

Extraire les éléments du PDF pouvant être utilisés dans le processus de recherche. Ces éléments peuvent inclure : texte, images, tableaux, etc.

### Partitionner les tableaux, le texte et les images du PDF.

In [ ]:
output_path = "./content/"
file_path = output_path + 'attention.pdf'

chunks = partition_pdf(
    filename=file_path,
    infer_table_structure=True,            # extraire les tableaux
    strategy="hi_res",                     # obligatoire pour inférer les tableaux

    extract_image_block_types=["Image"],   # ajouter 'Table' à la liste pour extraire les images des tableaux
    # image_output_dir_path=output_path,   # si None, les images et tableaux seront enregistrés en base64

    extract_image_block_to_payload=True,   # si True, extrait en base64 pour usage API

    chunking_strategy="by_title",          # ou 'basic'
    max_characters=10000,                  # par défaut 500
    combine_text_under_n_chars=2000,       # par défaut 0
    new_after_n_chars=6000,
)


In [ ]:
# Nous obtenons 2 types d'éléments à partir de la fonction partition_pdf
set([str(type(el)) for el in chunks])

In [ ]:
# Chaque CompositeElement contient plusieurs éléments liés entre eux.
# Cela facilite l'utilisation de ces éléments ensemble dans un pipeline RAG.

chunks[3].metadata.orig_elements

In [ ]:
# Voici à quoi ressemble une image extraite.
# Elle contient uniquement la représentation base64 car nous avons défini le paramètre extract_image_block_to_payload=True

elements = chunks[3].metadata.orig_elements
chunk_images = [el for el in elements if 'Image' in str(type(el))]
chunk_images[0].to_dict()

### Pour séparer les éléments extraits en tableaux, texte et images

In [ ]:
# Séparer les tableaux des textes
tableaux = []
textes = []

for chunk in chunks:
    # Vérifier si l'élément est un tableau
    if "Table" in str(type(chunk)):
        tableaux.append(chunk)

    # Vérifier si l'élément est un texte ou un élément composite
    if "CompositeElement" in str(type(chunk)):
        textes.append(chunk)

In [ ]:
# Obtenir les images en base64 des objets CompositeElement
def obtenir_images_base64(chunks):
    images_b64 = []
    for chunk in chunks:
        # Vérifier si l'élément est un CompositeElement
        if "CompositeElement" in str(type(chunk)):
            chunk_els = chunk.metadata.orig_elements
            for el in chunk_els:
                # Vérifier si l'élément est une image
                if "Image" in str(type(el)):
                    images_b64.append(el.metadata.image_base64)  # Ajouter l'image en base64
    return images_b64

# Appeler la fonction pour récupérer les images
images = obtenir_images_base64(chunks)

#### Vérifier à quoi ressemble l'image

In [ ]:
import base64
from IPython.display import Image, display

def afficher_image_base64(base64_code):
    # Décoder la chaîne base64 en binaire
    image_data = base64.b64decode(base64_code)
    # Afficher l'image
    display(Image(data=image_data))

# Afficher la première image en base64 dans la liste des images
afficher_image_base64(images[0])

Résumer les données
Créer un résumé de chaque élément extrait du PDF. Ce résumé sera vectorisé et utilisé dans le processus de récupération.

Résumés de texte et de tableaux
On pas besoin d'un modèle multimodal pour générer les résumés des tableaux et du texte. J'utiliserai des modèles open source disponibles sur Groq.

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
prompt_text = """
Vous êtes un assistant chargé de résumer des tableaux et des textes.
Donnez un résumé concis du tableau ou du texte.

Répondez uniquement avec le résumé, sans commentaire supplémentaire.
Ne commencez pas votre message par "Voici un résumé" ou quelque chose de similaire.
Donnez simplement le résumé tel quel.

Tableau ou extrait de texte : {element}
"""
prompt = ChatPromptTemplate.from_template(prompt_text)

# Chaîne de résumé
model = ChatGroq(temperature=0.5, model="llama-3.1-8b-instant")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [ ]:
# Résumer le texte
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 3})

# Résumer les tableaux
tables_html = [table.metadata.text_as_html for table in tables]
table_summaries = summarize_chain.batch(tables_html, {"max_concurrency": 3})

In [ ]:
text_summaries

### Résumé des images

Utilisation de GPT-4o-mini pour produire les résumés des images.

In [ ]:
%pip install -Uq langchain_openai

In [ ]:
from langchain_openai import ChatOpenAI

prompt_template = """Describe the image in detail. For context,
                  the image is part of a research paper explaining the transformers
                  architecture. Be specific about graphs, such as bar plots."""
messages = [
    (
        "user",
        [
            {"type": "text", "text": prompt_template},
            {
                "type": "image_url",
                "image_url": {"url": "data:image/jpeg;base64,{image}"},
            },
        ],
    )
]

prompt = ChatPromptTemplate.from_messages(messages)

chain = prompt | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()


# Résumer les images
image_summaries = chain.batch(images)


In [ ]:
image_summaries

In [ ]:
print(image_summaries[1])

## Charger les données et les résumés dans le vectorstore

### Création des vecteurs

In [ ]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever

# Le vectorstore à utiliser pour indexer les éléments enfants
vectorstore = Chroma(collection_name="multi_modal_rag", embedding_function=OpenAIEmbeddings())

# Le couche de stockage pour les documents parents
store = InMemoryStore()
id_key = "doc_id"

# Le récupérateur (vide pour commencer)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

### Charger les résumés et les lier aux données originales

In [ ]:
# Ajouter les textes
doc_ids = [str(uuid.uuid4()) pour _ dans textes]
summary_texts = [
    Document(page_content=summary, metadata={id_key: doc_ids[i]}) pour i, summary dans enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, textes)))

# Ajouter les tableaux
table_ids = [str(uuid.uuid4()) pour _ dans tableaux]
summary_tables = [
    Document(page_content=summary, metadata={id_key: table_ids[i]}) pour i, summary dans enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tableaux)))

# Ajouter les résumés d'images
img_ids = [str(uuid.uuid4()) pour _ dans images]
summary_img = [
    Document(page_content=summary, metadata={id_key: img_ids[i]}) pour i, summary dans enumerate(image_summaries)
]
retriever.vectorstore.add_documents(summary_img)
retriever.docstore.mset(list(zip(img_ids, images)))

### Vérification de la récupération

In [ ]:
# Recuperation
docs = retriever.invoke(
    "who are the authors of the paper?"
)

In [ ]:
for doc in docs:
    print(str(doc) + "\n\n" + "-" * 80)

## RAG pipeline

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from base64 import b64decode


def parse_docs(docs):
    """Split base64-encoded images and texts"""
    b64 = []
    text = []
    for doc in docs:
        try:
            b64decode(doc)
            b64.append(doc)
        except Exception as e:
            text.append(doc)
    return {"images": b64, "texts": text}


def build_prompt(kwargs):

    docs_by_type = kwargs["context"]
    user_question = kwargs["question"]

    context_text = ""
    if len(docs_by_type["texts"]) > 0:
        for text_element in docs_by_type["texts"]:
            context_text += text_element.text

    prompt_template = f"""
    Answer the question based only on the following context, which can include text, tables, and the below image.
    Context: {context_text}
    Question: {user_question}
    """

    prompt_content = [{"type": "text", "text": prompt_template}]

    if len(docs_by_type["images"]) > 0:
        for image in docs_by_type["images"]:
            prompt_content.append(
                {
                    "type": "image_url",
                    "image_url": {"url": f"data:image/jpeg;base64,{image}"},
                }
            )

    return ChatPromptTemplate.from_messages(
        [
            HumanMessage(content=prompt_content),
        ]
    )


chain = (
    {
        "context": retriever | RunnableLambda(parse_docs),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(build_prompt)
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

chain_with_sources = {
    "context": retriever | RunnableLambda(parse_docs),
    "question": RunnablePassthrough(),
} | RunnablePassthrough().assign(
    response=(
        RunnableLambda(build_prompt)
        | ChatOpenAI(model="gpt-4o-mini")
        | StrOutputParser()
    )
)

In [ ]:
response = chain.invoke(
    "What is the attention mechanism?"
)

print(response)

In [ ]:
response = chain_with_sources.invoke(
    "What is multihead?"
)

print("Response:", response['response'])

print("\n\nContext:")
for text in response['context']['texts']:
    print(text.text)
    print("Page number: ", text.metadata.page_number)
    print("\n" + "-"*50 + "\n")
for image in response['context']['images']:
    display_base64_image(image)

## References

- [LangChain Inspiration](https://github.com/langchain-ai/langchain/blob/master/cookbook/Semi_structured_and_multi_modal_RAG.ipynb?ref=blog.langchain.dev)
- [Multivector Storage](https://python.langchain.com/docs/how_to/multi_vector/)